In [26]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import sys
if '..' not in sys.path:
    sys.path.insert(0,'..')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [72]:
from sklearn.datasets import make_classification
X, y = make_classification(n_samples=125_000, n_features=30, n_informative=10, n_redundant=10, shuffle=False, random_state=42)
X = pd.DataFrame(X, columns=[f'feat_{idx}' for idx in range(X.shape[1])])
y = pd.Series(y)

from sklearn.model_selection import train_test_split
X_tr, X_ts, y_tr, y_ts = train_test_split(X, y, test_size=.2)
X_tr = X_tr.reset_index(drop=True)
X_ts = X_ts.reset_index(drop=True)
y_tr = y_tr.reset_index(drop=True)
y_ts = y_ts.reset_index(drop=True)

In [3]:
from ml_tools.optimization.classification import optimize_rf
optimize_rf(X_tr, y_tr)

-------------CROSS-VALIDATING MODELS-------------


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    2.2s
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    2.2s
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:    2.2s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    2.2s
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    2.2s
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:    2.2s
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed:    2.2s
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    2.2s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    2.2s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    2.2s
[Parallel(n_jobs=-1)]: Done  11 tasks      | elapsed:    2.2s
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:    2.2s


KeyboardInterrupt: 

In [73]:
from sklearn.ensemble import RandomForestClassifier
model_rf = RandomForestClassifier(n_estimators=200, class_weight='balanced', max_features=.5, min_samples_leaf=10, random_state=42, n_jobs=-1)

from ml_tools.evaluation.classification import evaluate
%time evaluate(model_rf, X_tr, y_tr, X_ts, y_ts, rfimp=True)

,0
Feature,
feat_15,1.337720e-01
feat_0,1.201163e-01
feat_9,1.196369e-01
feat_17,9.538214e-02
feat_5,8.842342e-02
feat_4,8.173159e-02
feat_3,6.977594e-02
feat_8,4.789077e-02
feat_18,4.426543e-02


CPU times: user 5min 8s, sys: 527 ms, total: 5min 8s
Wall time: 58.5 s


In [76]:
from ml_tools.feature_selection.permutation_importance import PermImpElimination
perm = PermImpElimination(model_rf, X_tr, y_tr)
perm.fit(X_ts, y_ts)

dropped: feat_21 -0.00026894994365595935
dropped: feat_26 -0.0003843258821519781
dropped: feat_20 -8.774943526878425e-05
dropped: feat_29 -0.0002717090180096817
dropped: feat_23 -6.247336739001921e-07
dropped: feat_28 -4.894598208654694e-06
dropped: feat_19 -7.829590217690815e-06


In [77]:
from sklearn.ensemble import RandomForestClassifier
model_rf = RandomForestClassifier(n_estimators=200, class_weight='balanced', max_features=.5, min_samples_leaf=10, random_state=42, n_jobs=-1)

from ml_tools.evaluation.classification import evaluate
%time evaluate(model_rf, perm.transform(X_tr), y_tr, perm.transform(X_ts), y_ts, rfimp=True)

,0
Feature,
feat_15,0.133144
feat_9,0.128611
feat_0,0.125975
feat_17,0.108380
feat_5,0.095704
feat_4,0.081735
feat_8,0.048676
feat_3,0.048552
feat_13,0.038742


CPU times: user 3min 42s, sys: 483 ms, total: 3min 43s
Wall time: 42.9 s
